<a href="https://colab.research.google.com/github/mshojaei77/gemma-3n-E4B-persin-qlora/blob/main/Gemma3N_(4B)_Persian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + "0.0.32.post2" if v == "2.8.0" else "0.0.29.post3"
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth

In [ ]:
%%capture
!pip install --no-deps --upgrade timm # Only for Gemma 3N

In [ ]:
from unsloth import FastModel
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template
from trl import SFTTrainer, SFTConfig

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
# 2. Load Model and Tokenizer (ONLY ONCE)
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3n-E4B-it",
    dtype = None,
    max_seq_length = 4000,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.8.9: Fast Gemma3N patching. Transformers: 4.55.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3N does not support SDPA - switching to eager!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.72G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.15G [00:00<?, ?B/s]

...

preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

In [ ]:
# 3. Add LoRA Adapters (ONLY ONCE)
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = False,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 16,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model.language_model` require gradients


In [ ]:
# 4. Load and Clean Data
dataset = load_dataset("mshojaei77/persian-gk", split="train")

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

def formatting_prompts_func(examples):
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False).removeprefix('<bos>') for convo in examples["messages"]]
   return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)
dataset = dataset.filter(lambda example: len(example.get("text", "").strip()) > 0)

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/2.97M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5897 [00:00<?, ? examples/s]

Map:   0%|          | 0/5897 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5897 [00:00<?, ? examples/s]

In [ ]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)

In [ ]:
dataset[100]

{'messages': [{'content': 'سلام! من می\u200cخوام از جزیره کتاب خرید کنم. میشه بگی چه نوع محصولاتی دارین؟',
   'role': 'user'},
  {'content': 'سلام! خوش اومدی! ما تو جزیره کتاب انواع خودکارهای رنگارنگ، دفتر یادداشت\u200cهای شیک، جامدادی\u200cهای جذاب، مدادرنگی\u200cهای باکیفیت، بازی\u200cهای فکری، ملزومات دانش\u200cآموزی و کتاب\u200cهای مختلف برای همه سنین داریم.',
   'role': 'assistant'},
  {'content': 'چه برندهایی رو موجود دارین؟ دنبال یه خودکار خوب می\u200cگردم.',
   'role': 'user'},
  {'content': 'ما از برندهای معتبر جهانی مثل فابرکاستل، استدلر، اشنایدر و زبرا محصولات داریم. این برندها به خاطر کیفیت بالا و طراحی خوبشون معروفن. خودکارهای این برندها انتخابای خیلی خوبی هستن.',
   'role': 'assistant'}],
 'text': '<start_of_turn>user\nسلام! من می\u200cخوام از جزیره کتاب خرید کنم. میشه بگی چه نوع محصولاتی دارین؟<end_of_turn>\n<start_of_turn>model\nسلام! خوش اومدی! ما تو جزیره کتاب انواع خودکارهای رنگارنگ، دفتر یادداشت\u200cهای شیک، جامدادی\u200cهای جذاب، مدادرنگی\u200cهای باکیفیت، بازی\u2

In [ ]:
def formatting_prompts_func(examples):
   convos = examples["messages"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False).removeprefix('<bos>') for convo in convos]
   return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/5897 [00:00<?, ? examples/s]

In [ ]:
dataset[100]["text"]

'<start_of_turn>user\nسلام! من می\u200cخوام از جزیره کتاب خرید کنم. میشه بگی چه نوع محصولاتی دارین؟<end_of_turn>\n<start_of_turn>model\nسلام! خوش اومدی! ما تو جزیره کتاب انواع خودکارهای رنگارنگ، دفتر یادداشت\u200cهای شیک، جامدادی\u200cهای جذاب، مدادرنگی\u200cهای باکیفیت، بازی\u200cهای فکری، ملزومات دانش\u200cآموزی و کتاب\u200cهای مختلف برای همه سنین داریم.<end_of_turn>\n<start_of_turn>user\nچه برندهایی رو موجود دارین؟ دنبال یه خودکار خوب می\u200cگردم.<end_of_turn>\n<start_of_turn>model\nما از برندهای معتبر جهانی مثل فابرکاستل، استدلر، اشنایدر و زبرا محصولات داریم. این برندها به خاطر کیفیت بالا و طراحی خوبشون معروفن. خودکارهای این برندها انتخابای خیلی خوبی هستن.<end_of_turn>\n'

In [ ]:
!pip install --upgrade wandb

In [ ]:
import wandb

# This will prompt you for your API key
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shojaei-dev (shojaei-dev-stanford-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        num_train_epochs = 1,
        learning_rate = 2e-5,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "wandb",
    ),
)


In [ ]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

'<bos><start_of_turn>user\nسلام! من می\u200cخوام از جزیره کتاب خرید کنم. میشه بگی چه نوع محصولاتی دارین؟<end_of_turn>\n<start_of_turn>model\nسلام! خوش اومدی! ما تو جزیره کتاب انواع خودکارهای رنگارنگ، دفتر یادداشت\u200cهای شیک، جامدادی\u200cهای جذاب، مدادرنگی\u200cهای باکیفیت، بازی\u200cهای فکری، ملزومات دانش\u200cآموزی و کتاب\u200cهای مختلف برای همه سنین داریم.<end_of_turn>\n<start_of_turn>user\nچه برندهایی رو موجود دارین؟ دنبال یه خودکار خوب می\u200cگردم.<end_of_turn>\n<start_of_turn>model\nما از برندهای معتبر جهانی مثل فابرکاستل، استدلر، اشنایدر و زبرا محصولات داریم. این برندها به خاطر کیفیت بالا و طراحی خوبشون معروفن. خودکارهای این برندها انتخابای خیلی خوبی هستن.<end_of_turn>\n'

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
12.592 GB of memory reserved.


# Let's train the model!

To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,897 | Num Epochs = 1 | Total steps = 738
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 15,482,880 of 7,865,461,072 (0.20% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,8.473800
2,8.087600
3,8.709500
4,8.744500
5,7.834900
6,8.453600
7,8.811400
8,8.511400
9,8.572000
10,8.279300


Step,Training Loss
1,8.473800
2,8.087600
3,8.709500
4,8.744500
5,7.834900
6,8.453600
7,8.811400
8,8.511400
9,8.572000
10,8.279300


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

4051.5526 seconds used for training.
67.53 minutes used for training.
Peak reserved memory = 12.592 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 85.422 %.
Peak reserved memory for training % of max memory = 0.0 %.


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model_save_name = "mshojaei77/gemma-3n-E4B-persin-lora-adaptors"

# Save the LoRA adapters
model.save_pretrained(model_save_name)
tokenizer.save_pretrained(model_save_name)

print(f"Model saved to '{model_save_name}'")```

In [ ]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "باغ تخت چه ویژگی‌هایی داره که اون رو به یکی از قدیمی‌ترین باغ‌های شیراز تبدیل کرده؟",}]
}]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
    tokenize = True,
    return_dict = True,
).to("cuda")

from transformers import TextStreamer
_ = model.generate(
    **inputs,
    max_new_tokens = 500, # Increase for longer outputs!
    temperature = 0.1, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

باغ تخت به خاطر قدمت و داشتن درختان بانو که از دوران صفویه به یادگار مانده، از باغ‌های قدیمی شیراز شناخته می‌شه. این باغ با درختان بانو و باغ‌های اطرافش، یه نمونه بارز از باغ‌های قدیمی شیرازیه که هنوز هم پابرجاست.<end_of_turn>


Because of RAM issue, i had to do the rest of opration (merging adaptors) in kaggle notebook